In [1]:
import os
os.chdir('..')

In [2]:
import pystache

data_folder = '../data/'
server_folder = './server/'
template_folder = './templates/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
directions = ['grad', 'hess', 'bfgs', 'hybrid']
types = ['LS', 'TR', 'LS-ABS', 'TR-ABS']

combinations = {}

# Add all combinations
for d in directions:
    for t in types:
        combinations[t + '_' + d] = {'type': t, 'direction': d}
        
# Remove the unfeasible ones
del combinations['TR_grad']
del combinations['TR-ABS_grad']
del combinations['LS_hybrid']
del combinations['TR_hybrid']

In [55]:
general_params = {
    'model': 'MNL',
    'repetitions': 20,
    'batch': 100,
    'scitas': False,
    'time': '24'}

In [11]:
general_params = {
    'model': 'Nested',
    'repetitions': 20,
    'batch': 100,
    'scitas': False,
    'time': '24'}

In [9]:
general_params = {
    'model': 'LPMC_DrivingCost',
    'repetitions': 20,
    'batch': 1000,
    'scitas': True,
    'time': '12'}

In [56]:
general_params = {
    'model': 'LPMC_RemoveRest',
    'repetitions': 20,
    'batch': 1000,
    'scitas': True,
    'time': '24'}

In [7]:
general_params = {
    'model': 'LPMC_Full',
    'repetitions': 20,
    'batch': 1000,
    'scitas': True,
    'time': '36'}

In [12]:
if not os.path.exists(server_folder + general_params['model']):
    os.makedirs(server_folder + general_params['model'])
    
infile = open(server_folder + general_params['model'] + '/run_all.sh', 'w+')    

prep_line = None
if general_params['scitas']:
    prep_line = 'sbatch {}.run\n'
else:
    prep_line = 'python {}.py\n'
    
lines = [prep_line.format(c) for c in combinations]

for l in lines:
    infile.write(l)

for c in combinations:
    
    params = general_params
    params['type'] = combinations[c]['type']
    params['direction'] = combinations[c]['direction']
    params['filename'] = c

    with open(template_folder + '/optimization_python.mustache', 'r') as infile:
        tmpl = infile.read()
        
    tmpl_filed = pystache.render(tmpl, params)
    
    with open(server_folder + params['model'] + '/' + params['filename'] + '.py', 'w') as outfile:
        outfile.write(tmpl_filed)
    
    if params['scitas']:
        with open(template_folder + '/optimization_scrum.mustache', 'r') as infile:
            tmpl = infile.read()
            
        tmpl_filed = pystache.render(tmpl, params)
    
        with open(server_folder + params['model'] + '/' + params['filename'] + '.run', 'w') as outfile:
            outfile.write(tmpl_filed)
        